# How to include output from another step in a SoS step

* **Difficulty level**: intermediate
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * Function `output_from(step)` refers to output from another `step`
  * `output_from(step)[name]` can be used to refer to named output from `step`
  

## Referring to named output from another step

As shown in the example from tutorial [How to use named output in data-flow style workflows](doc/user_guide/named_output.html), function `named_output` can be used to refer to named output from another step:

In [1]:
!rm -f data/DEG.csv
%run plot -v1

[global]
excel_file = 'data/DEG.xlsx'
csv_file = 'data/DEG.csv'
figure_file = 'output.pdf'

[convert]
input: excel_file
output: csv = _input.with_suffix('.csv')

run: expand=True
    xlsx2csv {_input} > {_output}

[plot]
input: named_output('csv')
output: figure_file

R: expand=True
    data <- read.csv('{_input}')
    pdf('{_output}')
    plot(data$log2FoldChange, data$stat)
    dev.off()

ERROR: No step to generate target data/DEG.xlsx, needed by 'convert'


One obvious limitation of `named_output()` is that the name has to be unique in the workflow. For example, in the following script where another step `test_csv` also gives its output a name `csv`, the workflow would fail due to ambiguity. This is usually not a concern with small workflows. However, when workflows get more and more complex, it is sometimes desired to anchor named output more precisely.

In [2]:
!rm -f data/DEG.csv
%run plot -v1

[global]
excel_file = 'data/DEG.xlsx'
csv_file = 'data/DEG.csv'
figure_file = 'output.pdf'

[convert]
input: excel_file
output: csv = _input.with_suffix('.csv')

run: expand=True
    xlsx2csv {_input} > {_output}

[test_csv]
input: excel_file
output: csv = f'{_input:n}_test.csv'

run: expand=True
    xlsx2csv {_input} | head -10 > {_output}
    
[plot]
input: named_output('csv')
output: figure_file

R: expand=True
    data <- read.csv('{_input}')
    pdf('{_output}')
    plot(data$log2FoldChange, data$stat)
    dev.off()

ERROR: Multiple steps convert, test_csv to generate target named_output("csv")


## Function  `output_from` <a id="output_from"></a>

 <div class="bs-callout bs-callout-primary" role="alert">
    <h4>Function <code>output_from(steps, group_by, ...)</code></h4>
    <p>Function <code>output_from</code> refers to the output of <code>step</code>. The returned the object is the complete output from <code>step</code> with its own sources and groups. Therefore,</p>
    <ul>
        <li>More than one steps can be specified as a list of step names</li>
        <li>Option <code>group_by</code> can be used to regroup the returned files</li>
        <li><code>output_from(step)[name]</code> refers to all output with source <code>name</code></li>
    </ul>
 </div>

Function `output_from` imports the output from one or more other steps. For example, in the following workflow `output_from(['step_10', 'step_20'])` takes the output from steps `step_10` and `step_20` as input.

In [3]:
%run -v0
[step_10]
output: 'a.txt'
_output.touch()

[step_20]
output: 'b.txt'
_output.touch()

[step_30]
input:  output_from(['step_10', 'step_20'])
print(_input)

[###] 3 steps processed (3 jobs completed)


The above example is a simple forward workflow with numerically numbered steps. In this case the parameters of `output_from` can be simplied to just the indexes (integers) so the workflow can be written as

In [4]:
%run -v0
[step_10]
output: 'a.txt'
_output.touch()

[step_20]
output: 'b.txt'
_output.touch()

[step_30]
input:  output_from([10, 20])
print(_input)

[###] 3 steps processed (1 job completed, 2 jobs ignored)


The source `steps` of `output_from(steps)` does not have to be limited to numerically-indexed steps. For example, the above example can be written as:

In [5]:
%run -v0
[A]
output: 'a.txt'
_output.touch()

[B]
output: 'b.txt'
_output.touch()

[default]
input:  output_from(['A', 'B'])
print(_input)

[###] 3 steps processed (2 jobs completed, 1 job ignored)


### `labels` of outputs returned from `output_from`

The `sources` of the files returned from `output_from()` is by default the names of the steps so you can refer to these files separately using the `_input[name]` syntax:

In [6]:
%run -v0
[A]
output: 'a.txt'
_output.touch()

[B]
output: 'b.txt'
_output.touch()

[default]
input:  output_from(['A', 'B'])
print(_input)
print(f'Output from A is {_input["A"]}')
print(f'Output from B is {_input["B"]}')

[###] 3 steps processed (1 job completed, 2 jobs ignored)


If the output has its own sources (names), the sources will be kept.

In [7]:
%run -v0
[A]
output: A_out = 'a.txt'
_output.touch()

[B]
output: B_out = 'b.txt'
_output.touch()

[default]
input:  output_from(['A', 'B'])
print(_input)
print(f'Output from A is {_input["A_out"]}')
print(f'Output from B is {_input["B_out"]}')

[###] 3 steps processed (3 jobs completed)


As usual, keyword arguments of the input statement override the `sources` of input files:

In [8]:
%run -v0
[step_10]
output: 'a.txt'
_output.touch()

[step_20]
output: 'b.txt'
_output.touch()

[step_30]
input:  s10=output_from(10), s20=output_from(20)
print(f'Output from step_10 is {_input["s10"]}')
print(f'Output from step_20 is {_input["s20"]}')

[###] 3 steps processed (2 jobs completed, 1 job ignored)


### Groups of output returned from `output_from`

Similar to the case with `named_output`, the returned object from `output_from()` keeps its original groups. For example, 

In [9]:
%run B -v0
[A]
input: for_each=dict(i=range(4))
output: f'a_{i}.txt'
_output.touch()

[B]
input: output_from('A')
output: _input.with_suffix('.bak')
print(f'Converting {_input} to {_output}')
_output.touch()

[##] 2 steps processed (8 jobs completed)


You can override the groups using the `group_by` option of `output_from`. 

In [10]:
%run B -v0
[A]
input: for_each=dict(i=range(4))
output: f'a_{i}.txt'
_output.touch()

[B]
input: output_from('A', group_by=2)
output: [x.with_suffix('.bak') for x in _input]
print(f'Converting {_input} to {_output}')
_output.touch()

[##] 2 steps processed (2 jobs completed, 4 jobs ignored)


Note that we used 
```
_input.with_suffix('.bak')
```
when `_input` contains only one filename and the above the statement is equivalent to
```
_input[0].with_suffix('.bak')
```
However, when `_input` contains more than one files, you will have to deal with them one by one as follows:
```
[x.with_suffix('.bak') for x in _input]
```

## Using `output_from` in place of `named_output`

Going back to our `convert`, `plot` example. When another step is added to have the same named output, it is no longer possible to use `named_output(name)`. In this case you can explicitly specify the step from which the named output is defined, and use

```
output_from(step)[name]
```
instead of
```
named_output(name)
```
as shown in the following example:

In [11]:
!rm -f data/DEG.csv
%run plot -v1

[global]
excel_file = 'data/DEG.xlsx'
csv_file = 'data/DEG.csv'
figure_file = 'output.pdf'

[convert]
input: excel_file
output: csv = _input.with_suffix('.csv')

run: expand=True
    xlsx2csv {_input} > {_output}

[test_csv]
input: excel_file
output: csv = f'{_input:n}_test.csv'

run: expand=True
    xlsx2csv {_input} | head -10 > {_output}
    
[plot]
input: output_from('convert')['csv']
output: figure_file

R: expand=True
    data <- read.csv('{_input}')
    pdf('{_output}')
    plot(data$log2FoldChange, data$stat)
    dev.off()

ERROR: No step to generate target data/DEG.xlsx, needed by 'convert'


Note that `output_from` is better than `named_output` for its ability to referring to a specific step, but is also worse than `named_output` for the same reason because it makes the workflow more difficult to maintain. We generally recommend the use of `named_output` for its simplicity.

## `output_from()` with skipped substeps

Function `output_from()` obtains outputs, actually substeps output from another step. There is, however, a case when a substep is skipped and leaves no output. In this case, the substep output is dicarded.

For example, when a substep in the step `A` of the following workflow is skipped, the result from `output_from('A')` contains only the output of valid substeps. 

In [12]:
%run -v0
[A]
input: for_each=dict(i=range(4))
output: f'output_{i}.txt'

skip_if(i == 2, 'Skip substep 2')
_output.touch()

[default]
input: output_from('A')
print(_input)

[##] 2 steps processed (4 jobs completed, 3 jobs ignored)


However, if you would like to keep consistent number of substeps across steps, you can handle get output from all substeps by using option `remove_empty_groups=False`.

In [13]:
%run -v0
[A]
input: for_each=dict(i=range(4))
output: f'output_{i}.txt'

skip_if(i == 2, 'Skip substep 2')
_output.touch()

[default]
input: output_from('A', remove_empty_groups=False)
print(_input)

[##] 2 steps processed (4 jobs completed, 3 jobs ignored)


## Output from a workflow

 <div class="bs-callout bs-callout-info" role="alert">
    <h4>Function <code>output_from(workflow_name)</code></h4>
    <p><code>output_from(workflow_name)</code> is equivalent to <code>output_from(workflow_name_index)</code> where <code>index</code> is the largest index of the workflow <code>workflow_name</code></p>
 </div>

Function `output_from` is usually used to refer the output of a specific step. However, similar to [target `sos_step` that can refer to a numerically indexed workflow](target_sos_step.html), `output_from` can also accept the name of the workflow and returns the output of the last step of the workflow.

For example, in the following workflow, `output_from('A')` is used to obtain the output of step `A_2`, which is the last step of the workflow `A`. Although `output_from('A')` is identical to `output_from('A_2')`, it frees you from specifying the index of the last step of the workflow, and is more intuitive to think `output_from('A')` as the output of the workflow.

In [14]:
%run -v0 

[A_1]
print(f'Running {step_name} to analye data')

[A_2]
output: 'result.txt'

print(f'Running {step_name} to generate result')
_output.touch()

[default]
input: output_from('A')

print(f'Input of the {step_name} is {_input}')

[###] 3 steps processed (3 jobs completed)


## Further reading
* [How to use named output in data-flow style workflows](named_output.html)